<a href="https://colab.research.google.com/github/MapleWolfe/Milestone_2/blob/Jai/TFrecord_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data extraction from TF records

## installs, imports, pre-sets

In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 8.24 MiB/s, done.
Resolving deltas: 100% (191/191), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.0 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.

In [2]:
from google.colab import drive
import zipfile
import tensorflow as tf
import pandas as pd
import numpy as np
import skimage
from scipy.ndimage import distance_transform_edt
import warnings
warnings.filterwarnings('ignore')
import cudf
import cupy as cp
import gc


## Loading TF records from google drive

In [ ]:
# let's mount the drive
drive.mount('/content/drive')

# let's look into the zip file stored in the google drive
wild_fire_file_path = '/content/drive/MyDrive/next_day_wildfire.zip'
wildfire_zip =  zipfile.ZipFile(wild_fire_file_path, 'r')
tf_record_file_names = wildfire_zip.namelist()

print('number of TF records:', len(tf_record_file_names))
print('file names of tf records within the zip:')
print(tf_record_file_names)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
number of TF records: 19
file names of tf records within the zip:
['next_day_wildfire_spread_eval_00.tfrecord', 'next_day_wildfire_spread_eval_01.tfrecord', 'next_day_wildfire_spread_test_00.tfrecord', 'next_day_wildfire_spread_test_01.tfrecord', 'next_day_wildfire_spread_train_00.tfrecord', 'next_day_wildfire_spread_train_01.tfrecord', 'next_day_wildfire_spread_train_02.tfrecord', 'next_day_wildfire_spread_train_03.tfrecord', 'next_day_wildfire_spread_train_04.tfrecord', 'next_day_wildfire_spread_train_05.tfrecord', 'next_day_wildfire_spread_train_06.tfrecord', 'next_day_wildfire_spread_train_07.tfrecord', 'next_day_wildfire_spread_train_08.tfrecord', 'next_day_wildfire_spread_train_09.tfrecord', 'next_day_wildfire_spread_train_10.tfrecord', 'next_day_wildfire_spread_train_11.tfrecord', 'next_day_wildfire_spread_train_12.tfrecord', 'next_day_wildfire_spread_

In [ ]:
# unzipping one file at a time
def one_file_unzip(tf_record_file_name, zipfile_variable):
  extracted_record_path = zipfile_variable.extract(tf_record_file_name)
  raw_dataset = tf.data.TFRecordDataset(extracted_record_path)
  return raw_dataset

# yielding out one record at a time
def extract_one_row(tf_record_dataset):
  for i, raw_record in enumerate(tf_record_dataset.take(tf_record_dataset.cardinality().numpy())):
    one_record_dict = {}
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

    for key, feature in example.features.feature.items():

      kind = feature.WhichOneof('kind')
      one_record_dict[key] = np.array(getattr(feature, kind).value).reshape(64,64)
    yield one_record_dict

## let's create features from all images

### feature description given by dataset maker

In [ ]:
# data variables

INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph',
                  'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']

OUTPUT_FEATURES = ['FireMask']


# underlying feature value ranges:
# (min_clip, max_clip, mean, standard deviation)

feature_description_dict = {
    # Elevation in m: between 0.1 percentile and 99.9 percentile
    'elevation': (0.0, 3141.0, 657.3003, 649.0147),

    # Palmer Drought Severity Index: between 0.1 percentile and 99.9 percentile
    'pdsi': (-6.12974870967865, 7.876040384292651, -0.0052714925, 2.6823447),

    #Vegetation index times 10,000: between -1 and 1
    'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677),

    # Precipitation in mm: between 0.0 and 99.9 percentile
    'pr': (0.0, 44.53038024902344, 1.7398051, 4.482833),

    # Specific humidity: between 0 and 1
    'sph': (0., 1., 0.0071658953, 0.0042835088),

    # Wind direction in degrees clockwise from north: between 0 and 360.
    'th': (0., 360.0, 190.32976, 72.59854),

    #Min temp: between 253.15 kelvin and 99.9 percentile
    'tmmn': (253.15, 298.94891357421875, 281.08768, 8.982386),

    #Max temp: between 253.15 kelvin and 99.9 percentile
    'tmmx': (253.15, 315.09228515625, 295.17383, 9.815496),

    # Wind speed in m/s: between 0. and 99.9 percentile
    'vs': (0.0, 10.024310074806237, 3.8500874, 1.4109988),

    # NFDRS fire danger index energy release component BTU's per square foot.
    # 0., 99.9 percentile
    'erc': (0.0, 106.24891662597656, 37.326267, 20.846027),

    # Population density: between 0 and 99.9 percentile
    'population': (0., 2534.06298828125, 25.531384, 154.72331),

    # We don't want to normalize the FireMasks.
    # 1 indicates fire, 0 no fire, -1 unlabeled data
    'PrevFireMask': (-1., 1., 0., 1.),
    'FireMask': (-1., 1., 0., 1.)
}


### Feature generation

In [ ]:
# lets define the min max scaling function
def min_max_scaling(array,min_val,max_val):
    scaled_array = np.clip((array - min_val) / (max_val - min_val), 0, 1)
    return scaled_array

# let's apply guassian smoothing
def gaussian_smoothing(image_array,sigma_val):
  smooth_array = skimage.filters.gaussian(image_array, sigma=1)
  return smooth_array

#lets get the rate of change and mean,
def local_pixel_features(image_array,radius_val):
  footprint = skimage.morphology.disk(radius_val)
  gradient_array = skimage.filters.rank.gradient(image_array, footprint)
  mean_array = skimage.filters.rank.mean(image_array, footprint)
  return gradient_array,mean_array

#use altitude edge to identify whether pixel is at a similar altitude as any pixel that has fire
def fire_pixel_shared_altitude(row_dict, normalized_array, previous_day_fire = 'PrevFireMask'):
  edges_array = skimage.feature.canny(normalized_array)
  inverted_edges_array = np.logical_not(edges_array).astype(int)
  edge_label_array = skimage.measure.label(inverted_edges_array)

  previous_fire = row_dict[previous_day_fire]
  fire_edge_labels = (edge_label_array*previous_fire)

  unique_regions_with_fire = np.unique(fire_edge_labels.flatten())
  non_zero_unique_regions = unique_regions_with_fire[unique_regions_with_fire != 0]

  fire_at_same_altitude = np.isin(edge_label_array, non_zero_unique_regions).astype(int)
  return fire_at_same_altitude

def distance_to_fire(row_dict,feature):
  # we need to clip the fire mask to account for -1 values (missing values where the satellite was unable to get a clear image)
  # for now we take them as no fire objects, however we will not be accounting for these pixels in our model.
  fire_mask_array = row_dict[feature].clip(0,1)
  inverted_mask_array = 1 - fire_mask_array
  distance_transform_array = distance_transform_edt(inverted_mask_array)
  return distance_transform_array


In [ ]:
# let's apply it on all features
def build_features(record_dict,min_max_dict,sigma_val,radius_val):
  feature_list = record_dict.keys()
  output_feature_dict = {}
  for a_feature in feature_list:
    if a_feature not in ['PrevFireMask','FireMask']:
     #min max scaling
     feature_min = min_max_dict[a_feature][0]
     feature_max = min_max_dict[a_feature][1]
     scaled_array = min_max_scaling(record_dict[a_feature],feature_min,feature_max)
     #guassian smoothing
     smoothen_array = gaussian_smoothing(scaled_array,sigma_val)

     #local pixel values: gradient values(rate of change), local mean val.
     gradient_array,mean_array = local_pixel_features(smoothen_array,radius_val)

     #lets now add these features to our output:
     output_feature_dict[a_feature+'_'+'scaled_smoothened_values'] = smoothen_array.flatten()
     output_feature_dict[a_feature+'_'+'local_gradient'] = gradient_array.flatten()
     output_feature_dict[a_feature+'_'+'local_mean'] = mean_array.flatten()

     #lets label pixels if they are at the same elevation (to account for cliffs/mountains/chasms) as the fire
     # here we aren't using smoothened array
    if a_feature == 'elevation':
      fire_at_altitude_array = fire_pixel_shared_altitude(record_dict, scaled_array)
      output_feature_dict['fire_at_similar_altitude'] = fire_at_altitude_array.flatten()
     #lets move are features into a dict.

    # get pixel eucledian distance from fire
    if a_feature == 'PrevFireMask':
      distance_array = distance_to_fire(record_dict,a_feature)
      output_feature_dict['PrevFireMask'] = record_dict[a_feature].flatten()
      output_feature_dict['distance_from_fire'] = distance_array.flatten()

    if a_feature == 'FireMask':
      output_feature_dict['FireMask'] = record_dict[a_feature].flatten()

  return output_feature_dict



In [ ]:
def make_df_save_csv(string_file_name,tf_record_names = tf_record_file_names, main_zip_file = wildfire_zip, feature_descriptions= feature_description_dict):
  first_write = True
  total_rows = 0
  image_id = 0
  large_df_list = []
  column_names = []
  csv_file_name = string_file_name + '.csv'
  for a_tf_record in tf_record_names:
    if string_file_name in a_tf_record:
      print('started tf record: ', a_tf_record)
      raw_dataset = one_file_unzip(a_tf_record, main_zip_file)
      row_extraction_generator = extract_one_row(raw_dataset)
      single_record_list = []
      image_count = 0

      for a_row in row_extraction_generator:
        all_features_dict_array = build_features(a_row,feature_descriptions,sigma_val=1,radius_val=3)
        column_names = all_features_dict_array.keys()
        image_id += 1
        image_count +=1
        image_number_array = np.full(4096, image_id)
        all_features_dict_array['image_id'] = image_number_array

        if image_count == 1:
          all_features_dataframe = cudf.DataFrame.from_dict(all_features_dict_array)
        else:
          single_row_df = cudf.DataFrame.from_dict(all_features_dict_array)
          all_features_dataframe = all_features_dataframe.append(single_row_df, ignore_index=True)

        if image_count >= 200:
          single_record_list.append(all_features_dataframe)
          image_count = 0

      if image_count % 200 != 0:
        single_record_list.append(all_features_dataframe)
        image_count = 0

      big_df = cudf.concat(single_record_list, ignore_index=True)
      pandas_big_df = big_df.to_pandas()
      total_rows += len(pandas_big_df)
      if first_write == True:
        pandas_big_df.to_csv(csv_file_name, mode='a', index=False, header=True)
        first_write = False
      else:
        pandas_big_df.to_csv(csv_file_name, mode='a', index=False, header=False)

        print('completed: ', a_tf_record)

  print('csv output is complete')
  print('output csv lenght',total_rows)
  print('output csv image count: ', total_rows/4096)
  print('number of expected images', image_id)
  return None

In [ ]:
%%time
print('we are making eval csv')
make_df_save_csv('eval')

we are making eval csv
started tf record:  next_day_wildfire_spread_eval_00.tfrecord
completed:  next_day_wildfire_spread_eval_00.tfrecord
started tf record:  next_day_wildfire_spread_eval_01.tfrecord
completed:  next_day_wildfire_spread_eval_01.tfrecord
csv output is complete
output csv lenght 7688192
output csv image count:  1877.0
number of expected images 1877
CPU times: user 5min 25s, sys: 17.8 s, total: 5min 43s
Wall time: 6min 11s


In [ ]:
%%time
print('we are making eval csv')
make_df_save_csv('test')

we are making eval csv
started tf record:  next_day_wildfire_spread_test_00.tfrecord
completed:  next_day_wildfire_spread_test_00.tfrecord
started tf record:  next_day_wildfire_spread_test_01.tfrecord
completed:  next_day_wildfire_spread_test_01.tfrecord
csv output is complete
output csv lenght 6918144
output csv image count:  1689.0
number of expected images 1689
CPU times: user 4min 55s, sys: 14.7 s, total: 5min 9s
Wall time: 5min 12s


In [ ]:
%%time
print('we are making eval csv')
make_df_save_csv('train')

we are making eval csv
started tf record:  next_day_wildfire_spread_train_00.tfrecord
completed:  next_day_wildfire_spread_train_00.tfrecord
started tf record:  next_day_wildfire_spread_train_01.tfrecord
completed:  next_day_wildfire_spread_train_01.tfrecord
started tf record:  next_day_wildfire_spread_train_02.tfrecord
completed:  next_day_wildfire_spread_train_02.tfrecord
started tf record:  next_day_wildfire_spread_train_03.tfrecord
completed:  next_day_wildfire_spread_train_03.tfrecord
started tf record:  next_day_wildfire_spread_train_04.tfrecord
completed:  next_day_wildfire_spread_train_04.tfrecord
started tf record:  next_day_wildfire_spread_train_05.tfrecord
completed:  next_day_wildfire_spread_train_05.tfrecord
started tf record:  next_day_wildfire_spread_train_06.tfrecord
completed:  next_day_wildfire_spread_train_06.tfrecord
started tf record:  next_day_wildfire_spread_train_07.tfrecord
completed:  next_day_wildfire_spread_train_07.tfrecord
started tf record:  next_day_wild